# Clustering Crypto

In [38]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [39]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [40]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [41]:
# Alternatively, use the provided csv file:

csv_path = pd.read_csv("Resources/crypto_data.csv")
cryptocompare_df = pd.DataFrame(csv_path)
print(list(cryptocompare_df.columns))


# Create a DataFrame

['Unnamed: 0', 'CoinName', 'Algorithm', 'IsTrading', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply']


### Data Preprocessing

In [42]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
columns = ['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']
cryptocompare_df.drop(columns=[column for column in cryptocompare_df.columns if column not in columns], inplace=True)
cryptocompare_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [43]:
# Keep only cryptocurrencies that are trading
cryptocompare_df = cryptocompare_df[cryptocompare_df.IsTrading !=False]
cryptocompare_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [44]:
# Keep only cryptocurrencies with a working algorithm
cryptocompare_df = cryptocompare_df[cryptocompare_df.Algorithm !=False]
set(cryptocompare_df.Algorithm)

{'1GB AES Pattern Search',
 '536',
 'Argon2',
 'Argon2d',
 'BLAKE256',
 'Blake',
 'Blake2S',
 'Blake2b',
 'C11',
 'Cloverhash',
 'Counterparty',
 'CryptoNight',
 'CryptoNight Heavy',
 'CryptoNight Heavy X',
 'CryptoNight-Lite',
 'CryptoNight-V7',
 'CryptoNight-lite',
 'Cryptonight-GPU',
 'Curve25519',
 'DPoS',
 'Dagger',
 'Dagger-Hashimoto',
 'ECC 256K1',
 'Equihash',
 'Equihash+Scrypt',
 'Equihash1927',
 'Ethash',
 'Exosis',
 'Green Protocol',
 'Groestl',
 'HMQ1725',
 'HybridScryptHash256',
 'IMesh',
 'Jump Consistent Hash',
 'Keccak',
 'Leased POS',
 'Lyra2RE',
 'Lyra2REv2',
 'Lyra2Z',
 'M7 POW',
 'Momentum',
 'Multiple',
 'NIST5',
 'NeoScrypt',
 'Ouroboros',
 'PHI1612',
 'POS 2.0',
 'POS 3.0',
 'PoS',
 'Progressive-n',
 'Proof-of-Authority',
 'Proof-of-BibleHash',
 'QUAIT',
 'QuBit',
 'Quark',
 'QuarkTX',
 'Rainforest',
 'SHA-256',
 'SHA-256 + Hive',
 'SHA-256D',
 'SHA-512',
 'SHA3',
 'SHA3-256',
 'Scrypt',
 'Scrypt-n',
 'Semux BFT consensus',
 'Shabal256',
 'Skein',
 'SkunkHash',
 

In [45]:
# Remove the "IsTrading" column
cryptocompare_df.drop(columns='IsTrading', inplace=True)
cryptocompare_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
1243,Super Zero,Ethash,PoW,NaN,1000000000
1244,UOS,SHA-256,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [46]:
# Remove rows with at least 1 null value
cryptocompare_df = cryptocompare_df[cryptocompare_df.TotalCoinsMined.isna()==False]
cryptocompare_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [47]:
# Remove rows with cryptocurrencies having no coins mined
cryptocompare_df.dropna(inplace=True)
cryptocompare_df = cryptocompare_df[cryptocompare_df.TotalCoinsMined > 0]
cryptocompare_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [48]:
# Drop rows where there are 'N/A' text values
for columns in cryptocompare_df.columns:
    cryptocompare_df = cryptocompare_df[cryptocompare_df.notna()]

cryptocompare_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [50]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_list = [name for name in cryptocompare_df.CoinName]
coin_name = pd.DataFrame(coin_list)
coin_name.index = cryptocompare_df.index

In [52]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
cryptocompare_df.drop(columns='CoinName', inplace=True)

In [54]:
# Create dummy variables for text features
X = pd.get_dummies(cryptocompare_df,columns=['Algorithm', 'ProofType'])

In [55]:
# Standardize data
data_scaler = StandardScaler()
cryptoscaler_data = data_scaler.fit_transform(X)
cryptoscaler_data

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Reducing Dimensions Using PCA

In [56]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
pca_crypto = pca.fit_transform(cryptoscaler_data)

In [58]:
# Create a DataFrame with the principal components data
df_pca_crypto = pd.DataFrame(data=pca_crypto, columns=['Components1', 'Components2', 'Components3'], index = cryptocompare_df.index)
df_pca_crypto.head()

,Components1,Components2,Components3
0,-0.300562,0.992798,-0.469011
2,-0.283967,0.992841,-0.469644
5,2.300749,1.456333,-0.672231
7,-0.150146,-1.298536,0.185700
8,-0.164960,-1.984312,0.352917


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [62]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    kmeans = KMeans(n_clusters=1, random_state=0)
    kmeans.fit(df_pca_crypto)
    inertia.append(kmeans.inertia_)

# Create the Elbow Curve using hvPlot
elbow = {"k": k, "inertia": inertia}
elbow_df = pd.DataFrame(elbow)
elbow_df.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [69]:
# Initialize the K-Means model
kmodel = KMeans(n_clusters=4, random_state=0)

# Fit the model
kmodel.fit(df_pca_crypto)
# Predict clusters
cluster_predict = kmodel.predict(df_pca_crypto)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
df_pca_crypto["Class"] = kmodel.labels_
df_cluster = pd.concat([cryptocompare_df, df_pca_crypto], axis="columns")
df_cluster['CoinName'] = coin_list
df_cluster.index = cryptocompare_df.index
df_cluster.Class.value_counts()

1    287
0    239
3      5
2      1
Name: Class, dtype: int64

### Visualizing Results

#### 3D-Scatter with Clusters

In [77]:
# Create a 3D-Scatter with the PCA data and the clusters

scatter_fig = px.scatter_3d(
    df_cluster,
    x="Components3",
    y="Components2",
    z="Components1",
    color="Class",
    symbol="Class",
    width=600,
    hover_name = "CoinName",
    hover_data = ["Algorithm"]
)
scatter_fig.show()


#### Table of Tradable Cryptocurrencies

In [80]:
# Table with tradable cryptos
tradable_crypto = df_cluster.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"])
tradable_crypto

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [81]:
# Print the total number of tradable cryptocurrencies
print(len(df_cluster.CoinName))

532


#### Scatter Plot with Tradable Cryptocurrencies

In [84]:
# Scale data to create the scatter plot
min_max = MinMaxScaler()
X = df_cluster[["TotalCoinsMined", "TotalCoinSupply"]]

scale_data = min_max.fit(X)
scale_transform = scale_data.transform(X)

df_cluster[["TotalCoinesMined", "TotalCoinSupply"]] = scale_transform
df_cluster

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,Components1,Components2,Components3,Class,CoinName,TotalCoinesMined
0,Scrypt,PoW/PoS,4.199995e+01,4.200000e-11,-0.300562,0.992798,-0.469011,1,42 Coin,0.000000e+00
2,Scrypt,PoW/PoS,1.055185e+09,5.320000e-04,-0.283967,0.992841,-0.469644,1,404Coin,1.065855e-03
5,X13,PoW/PoS,2.927942e+10,3.141593e-01,2.300749,1.456333,-0.672231,1,EliteCoin,2.957551e-02
7,SHA-256,PoW,1.792718e+07,2.100000e-05,-0.150146,-1.298536,0.185700,0,Bitcoin,1.810842e-05
8,Ethash,PoW,1.076842e+08,0.000000e+00,-0.164960,-1.984312,0.352917,0,Ethereum,1.087731e-04
...,...,...,...,...,...,...,...,...,...,...
1238,SHA-256,DPoS,2.000000e+09,2.000000e-03,2.508596,0.897835,-0.228458,1,ZEPHYR,2.020225e-03
1242,Scrypt,PoW/PoS,1.493105e+07,2.500000e-04,-0.298615,0.992658,-0.469059,1,Gapcoin,1.508199e-05
1245,CryptoNight,PoW,9.802226e+08,1.400223e-03,0.310501,-2.310050,0.318863,0,Beldex,9.901351e-04
1246,Equihash,PoW,7.296538e+06,2.100000e-05,-0.179202,-2.096678,0.371110,0,Horizen,7.370282e-06


In [85]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
df_cluster.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", hover_cols=["CoinName"])

:Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)